In [1]:
import configparser, csv, json, os, sys
import logging, importlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

%load_ext autoreload
%autoreload 2

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.ekfnavtools as ekf
import posarmctools.epsgtools as epsg
import posarmctools.posar as posar
import posarmctools.sbg as sbg
from posarmctools import getNotebookLogger

In [2]:
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [3]:
conf = posar.FlightParams("/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_23_flight.ini")
#epsg3xxx = epsg.epsg3857 # web mercator
epsg3xxx = epsg.epsg3948

In [4]:
pts, ptsEpsg, ptsDict = epsg.getReferencePoints(conf.refs, epsg3xxx)

In [5]:
xA, yA = ptsEpsg['p2']
xB, yB = ptsEpsg['p0']
xCorner, yCorner = ptsEpsg['corner']
p1_x, p1_y = ptsEpsg['p1']
p3_x, p3_y = ptsEpsg['p3']
dAB = ((xB - xA)**2 + (yB - yA)**2)**0.5
xAB = (xB - xA) / dAB
yAB = (yB - yA) / dAB
ux = (xAB, yAB)
uy = (-yAB, xAB)

In [6]:
dAB

349.66459565970047

In [7]:
Dy = 350
x0 = (xCorner - 2000 * ux[0] - Dy * uy[0], yCorner - 2000 * ux[1] - Dy * uy[1])
x1 = (xCorner + 2000 * ux[0] - Dy * uy[0], yCorner + 2000 * ux[1] - Dy * uy[1])

s0 = (xCorner - 1000 * ux[0] - Dy * uy[0], yCorner - 1000 * ux[1] - Dy * uy[1])
s1 = (xCorner + 1000 * ux[0] - Dy * uy[0], yCorner + 1000 * ux[1] - Dy * uy[1])

Dy = 500
x2 = (xCorner - 2000 * ux[0] - Dy * uy[0], yCorner - 2000 * ux[1] - Dy * uy[1])
x3 = (xCorner + 2000 * ux[0] - Dy * uy[0], yCorner + 2000 * ux[1] - Dy * uy[1])

s2 = (xCorner - 1000 * ux[0] - Dy * uy[0], yCorner - 1000 * ux[1] - Dy * uy[1])
s3 = (xCorner + 1000 * ux[0] - Dy * uy[0], yCorner + 1000 * ux[1] - Dy * uy[1])

In [8]:
plt.figure()

plt.plot(x0[0], x0[1], '.', label='x0')
plt.plot(x1[0], x1[1], '.', label='x1')
plt.plot(s0[0], s0[1], '.', label='s0')
plt.plot(s1[0], s1[1], '.', label='s1')

plt.plot(x0[0], x0[1], '.', label='x0')
plt.plot(x1[0], x1[1], '.', label='x1')
plt.plot(s0[0], s0[1], '.', label='s0')
plt.plot(s1[0], s1[1], '.', label='s1')

plt.plot(xCorner, yCorner, '.', label='corner')
plt.plot(xA, yA, '.', label='p2')
plt.plot(xB, yB, '.', label='p0')
plt.plot(p1_x, p1_y, '.', label='p1')
plt.plot(p3_x, p3_y, '.', label='p3')
plt.legend()
plt.grid()
plt.gca().set_aspect("equal")

In [9]:
longlat = epsg.epsgToWgs84LongLat(x0, epsg3xxx)
f"{longlat[1]:.5f}   {longlat[0]:.5f}"

'48.06559   -1.98388'

In [10]:
s4 = (48.06586, -2.00112)
s5 = (48.08265, -2.00112)

In [11]:
def writePoint(myDict, key, xy_or_ll, dtype="xy"):
    if dtype == "xy":
        latLong = epsg.epsgToWgs84LongLat(xy_or_ll, epsg3xxx)[::-1]
    elif dtype == "ll":
        latLong = xy_or_ll
    else:
        print(f"error, unknown dtype: {dtype}")
        
    myDict[key] = f"{latLong[0]:.5f}, {latLong[1]:.5f}"

In [12]:
config = configparser.ConfigParser()
config['points'] = {}

writePoint(config['points'], 'x0', x0, dtype="xy")
writePoint(config['points'], 'x1', x1, dtype="xy")
writePoint(config['points'], 'x2', x2, dtype="xy")
writePoint(config['points'], 'x3', x3, dtype="xy")

writePoint(config['points'], 's0', s0, dtype="xy")
writePoint(config['points'], 's1', s1, dtype="xy")
writePoint(config['points'], 's2', s2, dtype="xy")
writePoint(config['points'], 's3', s3, dtype="xy")

writePoint(config['points'], 's4', s4, dtype="ll")
writePoint(config['points'], 's5', s5, dtype="ll")

filename = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25_points_tmp.ini"
with open(filename, 'w') as configfile:
    print(f"write {filename}")
    config.write(configfile)

write /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25_points_tmp.ini
